In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import re
import bm25s  # Assuming this is a custom BM25 implementation


## eval function

In [66]:
def hits_k(gold_list, retrieved_list):
    if set(gold_list).issubset(set(retrieved_list)):
        return 1
    else:
        return 0

def soft_hits_k(gold_list, retrieved_list):
    return int(any(item in retrieved_list for item in gold_list))

def strict_hits_k(gold_list, retrieved_list):
    return int(set(gold_list).issubset(set(retrieved_list)))

def recall_k(gold_list, retrieved_list):
    return len(set(gold_list).intersection(set(retrieved_list))) / len(gold_list) if gold_list else 0

def mrr_k(gold_list, retrieved_list):
    for idx, item in enumerate(retrieved_list):
        if item in gold_list:
            return 1/(idx+1)
    return 0

## BM25

In [34]:
corpus[1]

'Brimonidine is an α2 agonist medication used to treat open-angle glaucoma, ocular hypertension, and rosacea. In rosacea it improves the redness. It is used as eye drops or applied to the skin.  It is also available in the fixed-dose combination medication brimonidine/timolol along with timolol maleate.\nCommon side effects when used in the eyes include itchiness, redness, and a dry mouth. Common side effects when used on the skin include redness, burning, and headaches. More significant side effects may include allergic reactions and low blood pressure. Use in pregnancy appears to be safe. When applied to the eye it works by decreasing the amount of aqueous humor made while increasing the amount that drains from the eye. When applied to the skin it works by causing blood vessels to contract.\nBrimonidine was patented in 1972 and came into medical use in 1996. It is available as a generic medication. In 2022, it was the 199th most commonly prescribed medication in the United States, wi

In [11]:

# 1. Corpus Creation (same as original)
def create_corpus(folder):
    corpus = []
    for file in folder.iterdir():
        with Path(file).open("r") as f:
            corpus.append(f.read())
    return corpus

corpus = create_corpus(Path("../../data/background_information_data/drug_data/Wiki/")) + \
         create_corpus(Path("../../data/background_information_data/drug_data/Wiki_complexified/")) + \
         create_corpus(Path("../../data/background_information_data/drug_data/SMILES/")) + \
         create_corpus(Path("../../data/background_information_data/protein_data/Wiki/"))

# 2. Initialize BM25
corpus_tokens = bm25s.tokenize(corpus)
retriever = bm25s.BM25(corpus=corpus)
retriever.index(corpus_tokens)
retriever.save("bm25s_index")

Split strings:   0%|          | 0/797 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/797 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/797 [00:00<?, ?it/s]

Finding newlines for mmindex:   0%|          | 0.00/362k [00:00<?, ?B/s]

In [47]:
hits_5 = []
hits_10 = []
hits_15 = []
recall = []
mrr = []

retriever = bm25s.BM25.load("bm25s_index", load_corpus=True)




In [48]:
retriever = bm25s.BM25.load("bm25s_index", load_corpus=True)

# Initialize metrics
hits5, hits10, hits15 = [], [], []
recall, mrr = [], []

# Run evaluation
for row in test_df.itertuples():
    query_tokens = bm25s.tokenize(row.Question)
    gold_docs = set()

    if "SMILES" in row.Question_Background:
        split = row.Question_Background.strip().split("\n")
        for i in range(0, len(split) - 1, 3):
            if "INTERACTION" not in split[i]:
                gold_docs.add(f"{split[i]}\n{split[i+1]}")
    else:
        gold_docs = [x.strip() for x in re.findall(r"BACKGROUND INFORMATION:(.*?)(?=DRUG|PROTEIN|\Z)", row.Question_Background, re.DOTALL)]

    # For Hits@5/10/15
    for k, hits_list in zip([5, 10, 15], [hits5, hits10, hits15]):
        retrieved_docs, _ = retriever.retrieve(query_tokens, k=k)
        retrieved_texts = [x["text"] for x in retrieved_docs[0]]
        hits_list.append(soft_hits_k(gold_docs, retrieved_texts))
        #hits_list.append(strict_hits_k(gold_docs, retrieved_texts))
    # Recall@5 and MRR@5 only
    retrieved_docs_5, _ = retriever.retrieve(query_tokens, k=5)
    retrieved_texts_5 = [x["text"] for x in retrieved_docs_5[0]]
    recall.append(recall_k(gold_docs, retrieved_texts_5))
    mrr.append(mrr_k(gold_docs, retrieved_texts_5))


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

...

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [46]:
# Report strict
print(f"Hits@5: {np.round(np.mean(hits5), 4)}")
print(f"Hits@10: {np.round(np.mean(hits10), 4)}")
print(f"Hits@15: {np.round(np.mean(hits15), 4)}")
print(f"Recall@5: {np.round(np.mean(recall), 4)}")
print(f"MRR@5: {np.round(np.mean(mrr), 4)}")

Hits@5: 0.1964
Hits@10: 0.2798
Hits@15: 0.2976
Recall@5: 0.3978
MRR@5: 0.5169


In [49]:
# Report
print(f"Hits@5: {np.round(np.mean(hits5), 4)}")
print(f"Hits@10: {np.round(np.mean(hits10), 4)}")
print(f"Hits@15: {np.round(np.mean(hits15), 4)}")
print(f"Recall@5: {np.round(np.mean(recall), 4)}")
print(f"MRR@5: {np.round(np.mean(mrr), 2)}")

Hits@5: 0.5774
Hits@10: 0.5833
Hits@15: 0.5833
Recall@5: 0.3978
MRR@5: 0.52


## DPR

### v3: new model base(multi-set)

In [67]:
from transformers import (
    DPRQuestionEncoder, DPRQuestionEncoderTokenizer,
    DPRContextEncoder, DPRContextEncoderTokenizer
)
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import pandas as pd


In [68]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
# 1. Load Models & Tokenizers
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-multiset-base").to(device)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-multiset-base")


def encode_query(queries):
    encoded = question_tokenizer(queries, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    encoded = {k: v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
        return question_encoder(**encoded).pooler_output 
# context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
# context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [72]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base").to(device)

def encode_corpus(corpus, batch_size=32):
    all_embeddings = []
    for i in tqdm(range(0, len(corpus), batch_size), desc="Encoding corpus"):
        batch = corpus[i:i + batch_size]
        embs = encode_context(batch)  # uses your function
        all_embeddings.append(embs)
    return torch.cat(all_embeddings, dim=0)
corpus_embeddings = encode_corpus(corpus)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mode

In [77]:
corpus_embeddings = encode_corpus(corpus)
corpus_texts = corpus  # raw text used for reranking

top_k_values = [5, 10, 15]
soft_hits, strict_hits, recall, mrr = {k: [] for k in top_k_values}, {k: [] for k in top_k_values}, {5: []}, {5: []}

for row in tqdm(test_df.itertuples(), total=len(test_df)):
    query_vec = encode_query(row.Question)
    query_vec = query_vec / query_vec.norm(dim=1, keepdim=True)
    corpus_norm = corpus_embeddings / corpus_embeddings.norm(dim=1, keepdim=True)
    scores = torch.matmul(query_vec, corpus_norm.T).squeeze(0).cpu().numpy()

    top_idx = np.argsort(-scores)[:max(top_k_values)]

    # Gold doc extraction
    gold_docs = set()
    if "SMILES" in row.Question_Background:
        split = row.Question_Background.strip().split("\n")
        for i in range(0, len(split) - 1, 3):
            if "INTERACTION" not in split[i]:
                gold_docs.add(f"{split[i]}\n{split[i+1]}")
    else:
        gold_docs = [x.strip() for x in re.findall(r"BACKGROUND INFORMATION:(.*?)(?=DRUG|PROTEIN|\Z)", row.Question_Background, re.DOTALL)]

    for k in top_k_values:
        top_docs = [corpus_texts[i] for i in top_idx[:k]]
        reranked_docs = rerank_cross_encoder(row.Question, top_docs)

        soft_hits[k].append(soft_hits_k(gold_docs, reranked_docs))
        strict_hits[k].append(strict_hits_k(gold_docs, reranked_docs))
        if k == 5:
            recall[5].append(recall_k(gold_docs, reranked_docs))
            mrr[5].append(mrr_k(gold_docs, reranked_docs))


100%|██████████| 168/168 [00:33<00:00,  4.95it/s]


In [78]:
# --- Print Results ---
for k in top_k_values:
    print(f"\nResults for k={k}:")
    print(f"SoftHits@{k}:   {np.mean(soft_hits[k]):.2f}")
    print(f"StrictHits@{k}: {np.mean(strict_hits[k]):.2f}")
    if k == 5:
        print(f"Recall@5:       {np.mean(recall[5]):.2f}")
        print(f"MRR@5:          {np.mean(mrr[5]):.2f}")


Results for k=5:
SoftHits@5:   0.15
StrictHits@5: 0.01
Recall@5:       0.06
MRR@5:          0.14

Results for k=10:
SoftHits@10:   0.18
StrictHits@10: 0.01

Results for k=15:
SoftHits@15:   0.24
StrictHits@15: 0.02


In [75]:
corpus_texts = corpus
top_k_values = [5, 10, 15]
soft_hits, strict_hits, recall, mrr = {k: [] for k in top_k_values}, {k: [] for k in top_k_values}, {5: []}, {5: []}

for row in tqdm(test_df.itertuples(), total=len(test_df)):
    query_vec = encode_query(row.Question)
    query_vec = query_vec / query_vec.norm(dim=1, keepdim=True)
    corpus_norm = corpus_embeddings / corpus_embeddings.norm(dim=1, keepdim=True)
    scores = torch.matmul(query_vec, corpus_norm.T).squeeze(0).cpu().numpy()

    top_idx = np.argsort(-scores)[:max(top_k_values)]

    # Gold doc extraction remains unchanged
    gold_docs = set()
    if "SMILES" in row.Question_Background:
        split = row.Question_Background.strip().split("\n")
        for i in range(0, len(split) - 1, 3):
            if "INTERACTION" not in split[i]:
                gold_docs.add(f"{split[i]}\n{split[i+1]}")
    else:
        gold_docs = [x.strip() for x in re.findall(r"BACKGROUND INFORMATION:(.*?)(?=DRUG|PROTEIN|\Z)", row.Question_Background, re.DOTALL)]

    for k in top_k_values:
        top_docs = [corpus_texts[i] for i in top_idx[:k]]
        soft_hits[k].append(soft_hits_k(gold_docs, top_docs))
        strict_hits[k].append(strict_hits_k(gold_docs, top_docs))
        if k == 5:
            recall[5].append(recall_k(gold_docs, top_docs))
            mrr[5].append(mrr_k(gold_docs, top_docs))


  0%|          | 0/168 [00:00<?, ?it/s]

100%|██████████| 168/168 [00:01<00:00, 109.83it/s]


In [74]:
# --- Print Results ---
for k in top_k_values:
    print(f"\nResults for k={k}:")
    print(f"SoftHits@{k}:   {np.mean(soft_hits[k]):.2f}")
    print(f"StrictHits@{k}: {np.mean(strict_hits[k]):.2f}")
    if k == 5:
        print(f"Recall@5:       {np.mean(recall[5]):.2f}")
        print(f"MRR@5:          {np.mean(mrr[5]):.2f}")


Results for k=5:
SoftHits@5:   0.15
StrictHits@5: 0.01
Recall@5:       0.06
MRR@5:          0.10

Results for k=10:
SoftHits@10:   0.18
StrictHits@10: 0.01

Results for k=15:
SoftHits@15:   0.24
StrictHits@15: 0.02


### v1: too low

In [62]:
from transformers import (
    DPRQuestionEncoder, DPRQuestionEncoderTokenizer,
    DPRContextEncoder, DPRContextEncoderTokenizer
)
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import pandas as pd


In [63]:

# 1. Load Models & Tokenizers
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

In [64]:
# 2. Encode Corpus (Contexts)
def encode_corpus(corpus, batch_size=32):
    corpus_embeddings = []
    for i in range(0, len(corpus), batch_size):
        batch = corpus[i:i+batch_size]
        inputs = context_tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
        embeddings = context_encoder(**inputs).pooler_output.detach().numpy()
        corpus_embeddings.append(embeddings)
    return np.vstack(corpus_embeddings)

corpus_embeddings = encode_corpus(corpus)

In [65]:
hits5, hits10, hits15 = [], [], []
recall, mrr = [], []

for row in test_df.itertuples():
    gold_docs = set()
    if "SMILES" in row.Question_Background:
        split = row.Question_Background.strip().split("\n")
        for i in range(0, len(split) - 1, 3):
            if "INTERACTION" not in split[i]:
                gold_docs.add(f"{split[i]}\n{split[i+1]}")
    else:
        gold_docs = [x.strip() for x in re.findall(r"BACKGROUND INFORMATION:(.*?)(?=DRUG|PROTEIN|\Z)", row.Question_Background, re.DOTALL)]
   
    # Encode Question
    question_input = question_tokenizer(row.Question, return_tensors="pt")
    question_embedding = question_encoder(**question_input).pooler_output.detach().numpy()
    
    # Retrieve Top-K Contexts
    scores = cosine_similarity(question_embedding, corpus_embeddings)[0]

    # For Hits@5/10/15
    for k, hits_list in zip([5, 10, 15], [hits5, hits10, hits15]):
        retrieved_indices = np.argsort(scores)[-k:][::-1]  # Top-k most similar
        retrieved_texts = [corpus[idx] for idx in retrieved_indices]
        hits_list.append(strict_hits_k(gold_docs, retrieved_texts))
    
    # Recall@5 and MRR@5 only
    retrieved_indices_5 = np.argsort(scores)[-5:][::-1]
    retrieved_texts_5 = [corpus[idx] for idx in retrieved_indices_5]
    recall.append(recall_k(gold_docs, retrieved_texts_5))
    mrr.append(mrr_k(gold_docs, retrieved_texts_5))
# 6. Print Results (identical format)
print(f"Hits@5: {np.mean(hits5):.4f}")
print(f"Hits@10: {np.mean(hits10):.4f}")
print(f"Hits@15: {np.mean(hits15):.4f}")
print(f"Recall@5: {np.mean(recall):.4f}")
print(f"MRR@5: {np.mean(mrr):.4f}")
    

Hits@5: 0.0000
Hits@10: 0.0060
Hits@15: 0.0179
Recall@5: 0.0620
MRR@5: 0.0737


In [23]:
# 6. Print Results (identical format)
print(f"Hits@5: {np.mean(hits5):.4f}")
print(f"Hits@10: {np.mean(hits10):.4f}")
print(f"Hits@15: {np.mean(hits15):.4f}")
print(f"Recall@5: {np.mean(recall):.4f}")
print(f"MRR@5: {np.mean(mrr):.4f}")
    

Hits@5: 0.1429
Hits@10: 0.2262
Hits@15: 0.2738
Recall@5: 0.0620
MRR@5: 0.0737


In [25]:
import torch
import pandas as pd
import numpy as np
import re
from transformers import DPRQuestionEncoder, DPRContextEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer
from tqdm import tqdm

# Load DPR models and tokenizers with their specific tokenizers
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
question_encoder.to(device)
context_encoder.to(device)

# Function to encode queries using the question encoder
def encode_query(queries, max_length=128):
    encoded = question_tokenizer(
        queries,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}
    
    with torch.no_grad():
        outputs = question_encoder(**encoded)
        embeddings = outputs.pooler_output
    
    return embeddings

# Function to encode contexts using the context encoder
def encode_context(contexts, max_length=128):
    encoded = context_tokenizer(
        contexts,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}
    
    with torch.no_grad():
        outputs = context_encoder(**encoded)
        embeddings = outputs.pooler_output
    
    return embeddings

# Function to calculate similarity scores between query and documents
def calculate_similarity(query_embedding, doc_embeddings):
    # Normalize embeddings
    query_embedding = query_embedding / query_embedding.norm(dim=1, keepdim=True)
    doc_embeddings = doc_embeddings / doc_embeddings.norm(dim=1, keepdim=True)
    
    # Calculate dot product for similarity
    similarity_scores = torch.matmul(query_embedding, doc_embeddings.t()).squeeze(0)
    
    return similarity_scores.cpu().numpy()


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

In [26]:

# --- Corpus encoding ---
corpus_texts = []  # List of original texts, same order as embeddings
print("Encoding corpus...")
corpus_embeddings = []
batch_size = 32
for i in tqdm(range(0, len(corpus), batch_size)):
    batch_texts = corpus[i:i + batch_size]
    corpus_texts.extend(batch_texts)
    batch_embeddings = encode_context(batch_texts)
    corpus_embeddings.append(batch_embeddings)
corpus_embeddings = torch.cat(corpus_embeddings, dim=0)

torch.save({
    "embeddings": corpus_embeddings,
    "texts": corpus_texts
}, "dpr_corpus_embeddings.pt")

# --- Evaluation ---
print("Starting evaluation...")
top_k_values = [5, 10, 15]
soft_hits, strict_hits, recall, mrr = {}, {}, {}, {}
for k in top_k_values:
    soft_hits[k], strict_hits[k], recall[k], mrr[k] = [], [], [], []

# Load encoded corpus
saved_data = torch.load("dpr_corpus_embeddings.pt")
corpus_embeddings = saved_data["embeddings"].to(device)
corpus_texts = saved_data["texts"]

for row in tqdm(test_df.itertuples()):
    query_embedding = encode_query([row.Question])
    similarity_scores = calculate_similarity(query_embedding, corpus_embeddings)

    # Process gold docs
    gold_docs = set()
    if "SMILES" in row.Question_Background:
        split = row.Question_Background.strip().split("\n")
        for i in range(0, len(split) - 1, 3):
            if "INTERACTION" not in split[i]:
                gold_docs.add(f"{split[i]}\n{split[i+1]}")
    else:
        gold_docs = [x.strip() for x in re.findall(
            r"BACKGROUND INFORMATION:(.*?)(?=DRUG|PROTEIN|\Z)",
            row.Question_Background, re.DOTALL)]

    max_k = max(top_k_values)
    top_indices = np.argsort(-similarity_scores)[:max_k]

    for k in top_k_values:
        k_indices = top_indices[:k]
        retrieved_docs = [corpus_texts[idx] for idx in k_indices]

        soft_hits[k].append(soft_hits_k(gold_docs, retrieved_docs))
        strict_hits[k].append(strict_hits_k(gold_docs, retrieved_docs))
        recall[k].append(recall_k(gold_docs, retrieved_docs))
        mrr[k].append(mrr_k(gold_docs, retrieved_docs))

# --- Report ---
for k in top_k_values:
    print(f"\nResults for k={k}:")
    print(f"SoftHits@{k}:   {np.round(np.mean(soft_hits[k]), 2)}")
    print(f"StrictHits@{k}: {np.round(np.mean(strict_hits[k]), 2)}")
    print(f"Recall@{k}:     {np.round(np.mean(recall[k]), 2)}")
    print(f"MRR@{k}:        {np.round(np.mean(mrr[k]), 2)}")

Encoding corpus...


100%|██████████| 25/25 [00:01<00:00, 14.24it/s]


Starting evaluation...


168it [00:01, 104.56it/s]


Results for k=5:
SoftHits@5:   0.1
StrictHits@5: 0.0
Recall@5:     0.04
MRR@5:        0.05

Results for k=10:
SoftHits@10:   0.2
StrictHits@10: 0.0
Recall@10:     0.08
MRR@10:        0.07

Results for k=15:
SoftHits@15:   0.25
StrictHits@15: 0.01
Recall@15:     0.11
MRR@15:        0.07


### v2: add reader reranking--still low

In [30]:
from transformers import (
    DPRQuestionEncoder, DPRContextEncoder, DPRReader,
    DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer, DPRReaderTokenizer
)

question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)
reader = DPRReader.from_pretrained("facebook/dpr-reader-single-nq-base").to(device)

# Tokenizers
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
c_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
r_tokenizer = DPRReaderTokenizer.from_pretrained("facebook/dpr-reader-single-nq-base")


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

config.json:   0%|          | 0.00/484 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-reader-single-nq-base were not used when initializing DPRReader: ['span_predictor.encoder.bert_model.pooler.dense.bias', 'span_predictor.encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRReader from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRReader from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRReaderTokenizer'.


In [35]:

# --- Encode entire corpus ---
def encode_corpus(corpus, batch_size=32):
    corpus_embeddings = []
    for i in tqdm(range(0, len(corpus), batch_size), desc="Encoding corpus"):
        batch = corpus[i:i + batch_size]
        encoded = c_tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            output = context_encoder(**encoded).pooler_output
        corpus_embeddings.append(output)
    return torch.cat(corpus_embeddings, dim=0)

# --- Encode single query ---
def encode_query(query):
    encoded = q_tokenizer(query, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        return question_encoder(**encoded).pooler_output

# --- Cosine similarity ---
def calculate_similarity(query_emb, context_embs):
    query_emb = query_emb / query_emb.norm(dim=1, keepdim=True)
    context_embs = context_embs / context_embs.norm(dim=1, keepdim=True)
    return torch.matmul(query_emb, context_embs.T).squeeze(0).cpu().numpy()

In [38]:
corpus_embeddings = encode_corpus(corpus)

# Save for reuse
torch.save({"embeddings": corpus_embeddings, "corpus": corpus}, "dpr_corpus.pt")

# --- Evaluation ---
data = torch.load("dpr_corpus.pt")
corpus_embeddings = data["embeddings"].to(device)
corpus = data["corpus"]

test_df = pd.read_csv("../../dataset_for_hf/test.csv")
top_k_values = [5, 10, 15]
soft_hits, strict_hits, recall, mrr = {k: [] for k in top_k_values}, {k: [] for k in top_k_values}, {5: []}, {5: []}

for row in tqdm(test_df.itertuples(), total=len(test_df)):
    query_emb = encode_query(row.Question)
    sim_scores = calculate_similarity(query_emb, corpus_embeddings)
    top_indices = np.argsort(-sim_scores)[:max(top_k_values)]

    # Gold doc extraction
    gold_docs = set()
    if "SMILES" in row.Question_Background:
        split = row.Question_Background.strip().split("\n")
        for i in range(0, len(split) - 1, 3):
            if "INTERACTION" not in split[i]:
                gold_docs.add(f"{split[i]}\n{split[i+1]}")
    else:
        gold_docs = [x.strip() for x in re.findall(
            r"BACKGROUND INFORMATION:(.*?)(?=DRUG|PROTEIN|\Z)",
            row.Question_Background, re.DOTALL)]

    for k in top_k_values:
        k_indices = top_indices[:k]
        top_k_docs = [corpus[i] for i in k_indices]
        dummy_titles = [""] * k  # DPRReader requires titles

        # Reader re-ranking
        reader_inputs = r_tokenizer(
            questions=[row.Question] * k,
            titles=dummy_titles,
            texts=top_k_docs,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        # reader_inputs = r_tokenizer(
        #     questions=[row.Question] * k,
        #     titles=dummy_titles,
        #     texts=top_k_docs,
        #     return_tensors="pt",
        #     padding=True,
        #     truncation=True
        # ).to(device)

        with torch.no_grad():
            reader_output = reader(**reader_inputs)
        relevance_scores = reader_output.relevance_logits.squeeze().cpu().numpy()
        reranked = np.argsort(-relevance_scores)
        reranked_docs = [top_k_docs[i] for i in reranked]

        # Metrics
        soft_hits[k].append(soft_hits_k(gold_docs, reranked_docs))
        strict_hits[k].append(strict_hits_k(gold_docs, reranked_docs))
        if k == 5:
            recall[5].append(recall_k(gold_docs, reranked_docs))
            mrr[5].append(mrr_k(gold_docs, reranked_docs))

# --- Print Results ---
for k in top_k_values:
    print(f"\nResults for k={k}:")
    print(f"SoftHits@{k}:   {np.mean(soft_hits[k]):.2f}")
    print(f"StrictHits@{k}: {np.mean(strict_hits[k]):.2f}")
    if k == 5:
        print(f"Recall@5:       {np.mean(recall[5]):.2f}")
        print(f"MRR@5:          {np.mean(mrr[5]):.2f}")


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the


Results for k=5:
SoftHits@5:   0.10
StrictHits@5: 0.00
Recall@5:       0.04
MRR@5:          0.06

Results for k=10:
SoftHits@10:   0.20
StrictHits@10: 0.00

Results for k=15:
SoftHits@15:   0.25
StrictHits@15: 0.01


## MedCPT

In [59]:
import torch
import pandas as pd
import numpy as np
import re
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load models & tokenizers
article_model = AutoModel.from_pretrained("ncbi/MedCPT-Article-Encoder").to(device)
article_tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Article-Encoder")

query_model = AutoModel.from_pretrained("ncbi/MedCPT-Query-Encoder").to(device)
query_tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Query-Encoder")


cross_model = AutoModelForSequenceClassification.from_pretrained("ncbi/MedCPT-Cross-Encoder").to(device)
cross_tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Cross-Encoder")
# Function to encode queries using MedCPT Query Encoder

In [57]:

# Encode corpus
def encode_articles(corpus, batch_size=8):
    article_embs, processed_corpus = [], []
    for i in tqdm(range(0, len(corpus), batch_size), desc="Encoding articles"):
        batch_texts = corpus[i:i+batch_size]
        titles = [""] * len(batch_texts)  # dummy title for fair eval
        inputs = article_tokenizer(list(zip(titles, batch_texts)), padding=True, truncation=True, return_tensors="pt", max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            emb = article_model(**inputs).last_hidden_state[:, 0, :]  # CLS token
        article_embs.append(emb)
        processed_corpus.extend(batch_texts)
    return torch.cat(article_embs), processed_corpus


def encode_query(query):
    if isinstance(query, str):
        query = [query]  # wrap in list for batch processing
    encoded = query_tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=64)
    encoded = {k: v.to(device) for k, v in encoded.items() if isinstance(v, torch.Tensor)}
    with torch.no_grad():
        output = query_model(**encoded)
        return output.last_hidden_state[:, 0, :]  # [CLS] token


# Cross encoder rerank
def rerank_cross_encoder(query, docs):
    pairs = [[query, doc] for doc in docs]
    inputs = cross_tokenizer(pairs, truncation=True, padding=True, return_tensors="pt", max_length=512).to(device)
    with torch.no_grad():
        scores = cross_model(**inputs).logits.squeeze(dim=1).cpu().numpy()
    reranked = np.argsort(-scores)
    return [docs[i] for i in reranked]

In [60]:

# Encode entire corpus
corpus_embs, corpus_texts = encode_articles(corpus)

# --- Evaluation ---
top_k_values = [5, 10, 15]
soft_hits, strict_hits, recall, mrr = {k: [] for k in top_k_values}, {k: [] for k in top_k_values}, {5: []}, {5: []}

for row in tqdm(test_df.itertuples(), total=len(test_df)):
    query_vec = encode_query(row.Question)
    query_vec = query_vec / query_vec.norm(dim=1, keepdim=True)
    corpus_norm = corpus_embs / corpus_embs.norm(dim=1, keepdim=True)
    scores = torch.matmul(query_vec, corpus_norm.T).squeeze(0).cpu().numpy()

    top_idx = np.argsort(-scores)[:max(top_k_values)]

    # Gold docs
    gold_docs = set()
    if "SMILES" in row.Question_Background:
        split = row.Question_Background.strip().split("\n")
        for i in range(0, len(split) - 1, 3):
            if "INTERACTION" not in split[i]:
                gold_docs.add(f"{split[i]}\n{split[i+1]}")
    else:
        gold_docs = [x.strip() for x in re.findall(r"BACKGROUND INFORMATION:(.*?)(?=DRUG|PROTEIN|\Z)", row.Question_Background, re.DOTALL)]

    for k in top_k_values:
        top_docs = [corpus_texts[i] for i in top_idx[:k]]
        reranked_docs = rerank_cross_encoder(row.Question, top_docs)

        soft_hits[k].append(soft_hits_k(gold_docs, reranked_docs))
        strict_hits[k].append(strict_hits_k(gold_docs, reranked_docs))
        if k == 5:
            recall[5].append(recall_k(gold_docs, reranked_docs))
            mrr[5].append(mrr_k(gold_docs, reranked_docs))

Encoding articles:   0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 168/168 [00:24<00:00,  6.81it/s]


In [61]:
for k in top_k_values:
    print(f"\nResults for k={k}:")
    print(f"SoftHits@{k}:   {np.mean(soft_hits[k]):.2f}")
    print(f"StrictHits@{k}: {np.mean(strict_hits[k]):.2f}")
    if k == 5:
        print(f"Recall@5:       {np.mean(recall[5]):.2f}")
        print(f"MRR@5:          {np.mean(mrr[5]):.2f}")


Results for k=5:
SoftHits@5:   0.49
StrictHits@5: 0.10
Recall@5:       0.27
MRR@5:          0.45

Results for k=10:
SoftHits@10:   0.56
StrictHits@10: 0.15

Results for k=15:
SoftHits@15:   0.57
StrictHits@15: 0.21
